# Load Packages 


In [ ]:
import numpy as np
import pandas as pd
import os
import glob
# For plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import normalize
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
%matplotlib inline
#TSNE
from sklearn.manifold import TSNE
# LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
import tensorflow as tf
from tensorflow import  keras
import tensorflow as tf
from tensorflow import  keras
from tensorflow.keras.layers import Dense, TimeDistributed, Input, Lambda, LSTM, RepeatVector, Masking
import tensorflow.keras.models
import tensorflow.keras.optimizers
import tensorflow.keras.datasets
from tensorflow.keras import backend as K
from keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model 

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')


Mounted at /content/gdrive


# Load Model

In [ ]:
latent_dim = 128

In [ ]:
encoder =  keras.models.load_model('/content/gdrive/MyDrive/Projet_long/encoder128')

# Load Dataset

In [ ]:
def read_npy_file(item):
    data = np.load(item.decode())
    return data.astype(np.float32)

# load the embeddings generated in mebdeding notebook
data_path = "/content/gdrive/MyDrive/Projet_long/embd"
file_list = [os.path.join(data_path, p) for p in sorted(os.listdir(data_path))]

dataset = tf.data.Dataset.from_tensor_slices(file_list)

dataset = dataset.map(lambda item: tuple(tf.numpy_function(read_npy_file, [item], [tf.float32,])))


# Load Dataframe

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Projet_long/df.csv').iloc[: , 1:]

In [ ]:
seq_fold = df['Fold class']

In [ ]:
fold_arra = np.array(seq_fold)

# Encoding 

In [ ]:
encode_seq =  encoder.predict(dataset)

# tSNE

In [ ]:
vae_tsne_2d_coords = TSNE(n_components=2).fit_transform(encode_seq[1])

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [ ]:
vae_tsne_2d_coords

In [ ]:
# save coordinates
np.save('/content/gdrive/MyDrive/Projet_long/vae_tsne_2d_coords', vae_tsne_2d_coords)

# LDA

In [ ]:
vae_LDA_2d_coords = LinearDiscriminantAnalysis(n_components=2).fit_transform(encode_seq[1], fold_arra) 

In [ ]:
# save coordinates
np.save('/content/gdrive/MyDrive/Projet_long/vae_LDA_2d_coords', vae_LDA_2d_coords)

# Add coords to the dataframe

In [ ]:
def edit_datframe(dataframe, coords):
  x_coord = coords[:, 0].tolist()
  y_coord = coords[:, 1].tolist()
  dataframe['x_coords'] = x_coord
  dataframe['y_coords'] = y_coord

  return dataframe

In [ ]:
df2 = edit_datframe(df, vae_tsne_2d_coords)

In [ ]:
df2.to_csv('/content/gdrive/MyDrive/Projet_long/dataframe/df2.csv')